In [2]:
import pandas as pd

import mysql.connector

from sqlalchemy import create_engine

from skimpy import skim

# 0- importation des données en df

In [3]:
username = 'postgres'
password = 'admin'
host = '127.0.0.1'  # ou l'adresse IP de votre serveur PostgreSQL
port = '5432'  # le port par défaut pour PostgreSQL est 5432
database = 'preventive_maintenance'

# Créer l'URL de connexion
db_url = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

# Créer le moteur SQLAlchemy
engine = create_engine(db_url)

# Se connecter à la base de données
connexion = engine.connect()


In [4]:
df_aero = pd.read_sql_query('''
                  SELECT * 
                  FROM aeronefs
                  ''', connexion)


In [5]:
df_composants = pd.read_sql_query('''
                  SELECT * 
                  FROM composants
                  ''', connexion)

In [6]:
df_degradations = pd.read_sql_query('''
                  SELECT * 
                  FROM degradations
                  ''', connexion)

In [7]:
df_logs_vols = pd.read_sql_query('''
                  SELECT * 
                  FROM logs_vols
                  ''', connexion)

In [8]:
connexion.close()

# 1- merge des dataframes

In [9]:
# merge du degradations et composants en left join avec degradations en left

df_merge_deg_compo = pd.merge(df_degradations, df_composants, how='inner', left_on='compo_concerned', right_on='ref_compo')

In [10]:
df_merge_deg_compo.head(1)

,clef,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement,ref_compo,categorie,aero,desc_compo,lifespan,taux_usure_actuel,cout
0,D001982CRJ700_6943SYSCRJ700-CRJ700_6943-292024...,D001982,CRJ700_6943,SYSCRJ700-CRJ700_6943-29,16.941998,2024-06-02,False,SYSCRJ700-CRJ700_6943-29,Composants Majeurs,CRJ700_6943,Système d'extinction d'incendie,11971,16.941998,10109


In [11]:
# faire un group by par categoriee et ref_aero
df_merge_deg_compo_gb = df_merge_deg_compo.groupby(['aero', 'categorie', 'measure_day']).agg({'usure_nouvelle': ['max', 'mean']}).reset_index()

# Aplatir les colonnes
df_merge_deg_compo_gb.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_merge_deg_compo_gb.columns.values]


In [12]:
# faire un pivot pour obtenir une seule ligne par avion
df_pivot = df_merge_deg_compo_gb.pivot_table(index=['aero', 'measure_day'], columns=['categorie'], values=['usure_nouvelle_max', 'usure_nouvelle_mean'], aggfunc='first').reset_index()

df_pivot.columns = ['_'.join([str(i) for i in col]).strip() for col in df_pivot.columns.values]


In [13]:
df_pivot['measure_day_'] = df_pivot['measure_day_'].astype('str')

In [14]:
# ajout d'une clé aero et mesure_day
df_pivot['cle'] = df_pivot.apply(lambda x: x['aero_'] + '_' + x['measure_day_'], axis=1)

In [15]:
df_logs_vols

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant,temp,temp_unit,pressure,pressure_unit,vibrations,vibrations_unit
0,V06783026,A320_6242,2024-06-04,6.2,"{'temp': '-14.7°C', 'pressure': '1009.1 hPa', ...",1,-14.7,°C,1009.1,hPa,0.015340,m/s²
1,V06810954,A380_5199,2024-06-04,2.1,"{'temp': '2.9°C', 'pressure': '1015.8 hPa', 'v...",0,2.9,°C,1015.8,hPa,2.317764,m/s²
2,V05201226,A350_3122,2024-06-04,7.7,"{'temp': '8.0°C', 'pressure': '934.2 hPa', 'vi...",0,8.0,°C,934.2,hPa,1.999378,m/s²
3,V05582404,B747_3165,2024-06-04,8.3,"{'temp': '7.3°C', 'pressure': '999.9 hPa', 'vi...",0,7.3,°C,999.9,hPa,0.875383,m/s²
4,V01713095,A350_6452,2024-06-04,9.8,"{'temp': '-11.4°C', 'pressure': '946.7 hPa', '...",0,-11.4,°C,946.7,hPa,1.551531,m/s²
...,...,...,...,...,...,...,...,...,...,...,...,...
5036,V04286711,B767_2790,2024-07-22,7.4,"{'temp': '4.1°C', 'pressure': '895.8 hPa', 'vi...",0,4.1,°C,895.8,hPa,0.944693,m/s²
5037,V09194923,E170_0081,2024-07-23,6.6,"{'temp': '-10.0°C', 'pressure': '932.0 hPa', '...",0,-10.0,°C,932.0,hPa,3.773418,m/s²
5038,V02111126,A330_1199,2024-07-23,2.4,"{'temp': '9.9°C', 'pressure': '968.0 hPa', 'vi...",0,9.9,°C,968.0,hPa,1.499379,m/s²
5039,V06239425,A350_1689,2024-07-23,4.5,"{'temp': '10.9°C', 'pressure': '1024.4 hPa', '...",0,10.9,°C,1024.4,hPa,4.637732,m/s²


In [18]:
df_logs_vols['jour_vol'] = df_logs_vols['jour_vol'].astype('str')

In [19]:
# ajout d'une clé aero et mesure_day sur le df_logs_vols
df_logs_vols['cle'] = df_logs_vols.apply(lambda x: x['aero_linked'] + '_' + x['jour_vol'], axis=1)

In [20]:
# merge de df_log_vol et df_pivot

df_merge_pivot_logvol = pd.merge(df_pivot, df_logs_vols, how='inner', left_on='cle', right_on='cle')

In [22]:
# merge de df_aero et df_merge_pivot_logvol

df_merge_tot = pd.merge(df_merge_pivot_logvol, df_aero, how='inner', left_on='aero_linked', right_on='ref_aero')

In [24]:
df_merge_tot.shape

(5016, 27)

In [25]:
del df_aero, df_logs_vols, df_merge_pivot_logvol, df_degradations, df_composants

# 2- Nettoayage du df

In [26]:
# df_merge_tot.columns

Index(['aero_', 'measure_day_', 'usure_nouvelle_max_Composants Critiques',
       'usure_nouvelle_max_Composants Majeurs',
       'usure_nouvelle_max_Composants Secondaires',
       'usure_nouvelle_mean_Composants Critiques',
       'usure_nouvelle_mean_Composants Majeurs',
       'usure_nouvelle_mean_Composants Secondaires', 'cle', 'ref_vol',
       'aero_linked', 'jour_vol', 'time_en_air', 'sensor_data', 'etat_voyant',
       'temp', 'temp_unit', 'pressure', 'pressure_unit', 'vibrations',
       'vibrations_unit', 'ref_aero', 'type_model', 'debut_service',
       'last_maint', 'en_maintenance', 'end_maint'],
      dtype='object')

In [28]:
# suppression des colonne n'apportant pas d'informations
df_merge_tot = df_merge_tot.drop(columns=[
                            'cle', 'ref_vol',
                            'aero_linked', 'jour_vol', 'sensor_data', 
                            'temp_unit', 'pressure_unit',
                            'vibrations_unit', 'ref_aero', 
                            'end_maint'])

In [29]:
pd.set_option("display.max_columns", None)

In [ ]:
# skim(df_merge_total)

In [31]:
# modification du type pour date 
df_merge_total['debut_service'] = pd.to_datetime(df_merge_total['debut_service'], format='%Y-%m-%d')
df_merge_total['last_maint'] = pd.to_datetime(df_merge_total['last_maint'], format='%Y-%m-%d')
df_merge_total['measure_day'] = pd.to_datetime(df_merge_total['measure_day_'], format='%Y-%m-%d')

In [32]:
# modif type category
df_merge_total['etat_voyant'] = df_merge_total['etat_voyant'].astype('category')  

In [33]:
# Ajout d'une colonne temps de vol cumulé depuis la derniere maintenance

# Assumons que df_merge_total est le DataFrame fourni et qu'il contient les colonnes 'temps_de_vol', 'last_maint', 'jour_vol', et 'ref_aero'
# Trier le DataFrame par avion et par date de vol
df_merge_total = df_merge_total.sort_values(by=['aero_', 'measure_day']).reset_index(drop=True)

# Initialiser la colonne 'temps_de_vol_cumule' avec des zéros
df_merge_total['temps_de_vol_cumule'] = 0

# Parcourir chaque avion
for avion in df_merge_total['aero_'].unique():
    # Filtrer les données pour l'avion actuel
    df_avion = df_merge_total[df_merge_total['aero_'] == avion]
    
    # Variable pour stocker le temps de vol cumulé
    temps_de_vol_cumule = 0
    
    # Parcourir chaque ligne du DataFrame filtré pour l'avion actuel
    for i in df_avion.index:
        # Si la maintenance a eu lieu (en supposant que 'last_maint' est un booléen ou indicatif)
        if df_merge_total.loc[i, 'last_maint']:
            # Réinitialiser le temps de vol cumulé
            temps_de_vol_cumule = 0
        
        # Ajouter le temps de vol actuel au temps de vol cumulé
        temps_de_vol_cumule += df_merge_total.loc[i, 'time_en_air']
        
        # Mettre à jour la colonne 'temps_de_vol_cumule'
        df_merge_total.loc[i, 'temps_de_vol_cumule'] = temps_de_vol_cumule


        # affichage de la progression en %
        # print(f"Progression pour l'avion {avion}: {(i+1)/len(df_merge_total)*100:.2f}%")




C:\Users\dimle\AppData\Local\Temp\ipykernel_22920\2752787940.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_merge_total.loc[i, 'temps_de_vol_cumule'] = temps_de_vol_cumule


In [34]:
df_merge_total.head(3)

,aero_,measure_day_,usure_nouvelle_max_Composants Critiques,usure_nouvelle_max_Composants Majeurs,usure_nouvelle_max_Composants Secondaires,usure_nouvelle_mean_Composants Critiques,usure_nouvelle_mean_Composants Majeurs,usure_nouvelle_mean_Composants Secondaires,time_en_air,etat_voyant,temp,pressure,vibrations,type_model,debut_service,last_maint,en_maintenance,measure_day,temps_de_vol_cumule
0,A320_1884,2024-06-04,61.126660,60.781740,59.684620,28.157460,32.796355,29.171363,9.8,0,4.3,907.3,1.763677,A320,2015-12-26,2024-06-01,False,2024-06-04,9.8
1,A320_1884,2024-06-05,62.128398,61.547122,60.747186,29.045238,33.699044,30.416709,7.4,1,-24.3,905.9,3.428686,A320,2015-12-26,2024-06-01,False,2024-06-05,7.4
2,A320_1884,2024-06-08,62.466823,61.805697,61.106161,29.345163,34.004007,30.837434,2.5,0,-8.4,962.5,1.290613,A320,2015-12-26,2024-06-01,False,2024-06-08,2.5


In [ ]:
# skim(df_merge_total)

In [35]:
df_merge_total.to_parquet('data_preclean_avion.parquet')